<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/TF_trainer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/ashishkrb7/Object-Detection-TF-data-preparation/blob/main/TF_trainer.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
%reset -f

# Integrate Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/MyDrive'

# Dependencies installation, environment setup and pull TF API from github

In [ ]:
%cd /content
 
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
 
!pip install -q Cython contextlib2 pillow lxml matplotlib wget
 
!pip install -q pycocotools

!python -m pip install -q Cython pandas tf-slim lvis 

!python -m pip install -q tensorflow-addons

!python -m pip install -q tensorflow==1.14

import os
if not os.path.exists('/content/gdrive/MyDrive/Tensorflow'):os.makedirs('/content/gdrive/MyDrive/Tensorflow')
%cd '/content/gdrive/MyDrive/Tensorflow'
!git clone https://github.com/tensorflow/models.git

%cd '/content/gdrive/MyDrive/Tensorflow/models/research'
!protoc object_detection/protos/*.proto --python_out=.

import os
# Adding path in environment variable help us to use TF API as library
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/Tensorflow/models/research/:/content/gdrive/MyDrive/Tensorflow/models/research/slim/'
 
!python object_detection/builders/model_builder_test.py

# Remove unnecessary files and folder from the directory

In [ ]:
# import shutil,os
# shutil.rmtree("/content/gdrive/MyDrive/Tensorflow/models")
# os.remove("/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/exporter_main_v2.py")

# cd into the TensorFlow directory in your Google Drive and download models

        TensorFlow
        ├───scripts
        │   └───preprocessing
        │     └───generate_tfrecord.py 
        │     └───xml_to_csv.py 
        └───workspace
            └───training_demo
                ├───annotations
                │   └───label_map.pbtxt 
                ├───exported-models
                ├───images
                │   ├───test
                │   │     └───test images with corresponding XML files (put zip file, code will do rest of the work)
                │   └───train
                │         └───train images with corresponding XML files (put zip file, code will do rest of the work)
                ├───models
                │   └───my_ssd_resnet50_v1_fpn
                │     └───pipeline.config
                └───pre-trained-models
                    └───ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

In [ ]:
%cd '/content/gdrive/MyDrive'

In [1]:
repo_dir_path='.'

In [2]:
# create directory
if not os.path.exists(repo_dir_path+'/Tensorflow/scripts/preprocessing'):os.makedirs(repo_dir_path+'/Tensorflow/scripts/preprocessing')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/annotation'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/annotation')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/exported-models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/exported-models')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/images/train'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/images/train')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/images/test'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/images/test')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/models')
if not os.path.exists(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models'):os.makedirs(repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models')

# Load files

In [ ]:
# Load TF-recorder generator
import urllib.request
url_generate_tfrecord = "https://raw.githubusercontent.com/ashishkrb7/Object-Detection-TF-data-preparation/main/generate_tfrecord.py"
url_xml_to_csv = "https://raw.githubusercontent.com/ashishkrb7/Object-Detection-TF-data-preparation/main/xml_to_csv.py"
filename, headers = urllib.request.urlretrieve(url_generate_tfrecord, filename=repo_dir_path+"/Tensorflow/scripts/preprocessing/generate_tfrecord.py")
filename, headers = urllib.request.urlretrieve(url_xml_to_csv, filename=repo_dir_path+"/Tensorflow/scripts/preprocessing/xml_to_csv.py")

In [ ]:
# Load train and test dataset in zip formate to respective folder
from google.colab import files
%cd {repo_dir_path+'/Tensorflow/workspace/training_demo/images/train'}
upload=files.upload()

In [ ]:
!unzip train.zip
!rm train.zip

In [ ]:
%cd {'/content/gdrive/My Drive/Tensorflow/workspace/training_demo/images/test'}
upload=files.upload()

In [ ]:
!unzip test.zip
!rm test.zip

In [ ]:
%cd '/content'

In [ ]:
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/xml_to_csv.py -i gdrive/MyDrive/Tensorflow/workspace/training_demo/images/train -o gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train_labels.csv -l gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation
 
# Convert test folder annotation xml files to a single csv.
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/xml_to_csv.py -i gdrive/MyDrive/Tensorflow/workspace/training_demo/images/test -o gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test_labels.csv
 
# Generate `train.record`
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/generate_tfrecord.py --csv_input=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train_labels.csv --output_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train.record --img_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/images/train --label_map gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/label_map.pbtxt
 
# Generate `test.record`
!python gdrive/MyDrive/Tensorflow/scripts/preprocessing/generate_tfrecord.py --csv_input=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test_labels.csv --output_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test.record --img_path=gdrive/MyDrive/Tensorflow/workspace/training_demo/images/test --label_map gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/label_map.pbtxt


In [ ]:
test_record_fname = '/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/test.record'
train_record_fname = '/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/train.record'
label_map_pbtxt_fname = '/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/label_map.pbtxt'

In [ ]:
inputDir='/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/annotation/'
%cd {inputDir}
!zip -r data.zip {inputDir}

In [ ]:
from google.colab import files
files.download('data.zip')

# Download pre-trained model

To select the model for TF-1 [click here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)

Reference:
-   [TF1](https://towardsdatascience.com/object-detection-by-tensorflow-1-x-5a8cb72c1c4b)
-   [TF2](https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc)

In [ ]:
%cd '/content/gdrive/MyDrive/'

In [ ]:
model_name='ssd_inception_v2_coco_2018_01_28'

In [ ]:
if not os.path.exists(repo_dir_path+f'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}'):os.makedirs(repo_dir_path+f'/Tensorflow/workspace/training_demo/models/{model_name}')

In [ ]:
import wget
fname=wget.download(f'http://download.tensorflow.org/models/object_detection/{model_name}.tar.gz')
import shutil
shutil.unpack_archive(fname,repo_dir_path+'/Tensorflow/workspace/training_demo/pre-trained-models')
!rm {fname}

In [ ]:
if not os.path.exists(repo_dir_path+f'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}'):os.makedirs(repo_dir_path+f'/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}')

In [ ]:
# Copy pipline.config file. This file need to modify based on dataset
shutil.copy(f'./Tensorflow/workspace/training_demo/pre-trained-models/{model_name}/pipeline.config',f'./Tensorflow/workspace/training_demo/models/{model_name}')

# Configuring a Training Pipeline

In [ ]:
%cd '/content/gdrive/MyDrive/Tensorflow/models/research'

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re
file_resize_height= 800
file_resize_width= 600
num_classes = get_num_classes(label_map_pbtxt_fname)
num_steps=1000
batch_size=50
num_hard_examples= 100
fine_tune_checkpoint=f"/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/pre-trained-models/{model_name}/model.ckpt"

In [ ]:
configFile = "https://raw.githubusercontent.com/ashishkrb7/Object-Detection-TF-data-preparation/ssd_inception_v2_coco.config"
filename, headers = urllib.request.urlretrieve(configFile, filename=f'/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/models/{model_name}/ssd_inception_v2_coco.config")

In [ ]:
pipeline_fname1=f'/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/models/{model_name}/pipeline.config'
pipeline_fname2=f'/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/models/{model_name}/ssd_inception_v2_coco.config'

In [ ]:
with open(pipeline_fname1) as f:
    s = f.read()

with open(pipeline_fname1, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
 
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)
 
    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
 
    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Image height
    s = re.sub('height: [0-9]+',
               'height: {}'.format(file_resize_height), s)
    
    # Image width
    s = re.sub('width: [0-9]+',
               'width: {}'.format(file_resize_width), s)
    
    # num_hard_examples
    s = re.sub('num_hard_examples: [0-9]+',
               'num_hard_examples: {}'.format(num_hard_examples), s)   
    f.write(s)
  

In [ ]:
with open(pipeline_fname2) as f:
    s = f.read()

with open(pipeline_fname2, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
 
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)
 
    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
 
    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Image height
    s = re.sub('height: [0-9]+',
               'height: {}'.format(file_resize_height), s)
    
    # Image width
    s = re.sub('width: [0-9]+',
               'width: {}'.format(file_resize_width), s)
    
    # num_hard_examples
    s = re.sub('num_hard_examples: [0-9]+',
               'num_hard_examples: {}'.format(num_hard_examples), s)   
    f.write(s)
  

In [ ]:
!cat {pipeline_fname2}

# Training setup

## Setup verification

In [ ]:
%cd '/content/gdrive/MyDrive/Tensorflow/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
import shutil
shutil.copy('/content/gdrive/MyDrive/Tensorflow/models/research/object_detection/builders/model_builder_tf1_test.py','/content/gdrive/MyDrive/Tensorflow/models/research/')
shutil.copy('/content/gdrive/MyDrive/Tensorflow/models/research/object_detection/model_main.py','/content/gdrive/MyDrive/Tensorflow/workspace/training_demo')
shutil.copy('/content/gdrive/MyDrive/Tensorflow/models/research/object_detection/exporter.py','/content/gdrive/MyDrive/Tensorflow/workspace/training_demo')

In [ ]:
%cd '/content/gdrive/MyDrive/Tensorflow/models/research/'
!python model_builder_tf1_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

# Start Tensorboard

In [ ]:
!python -m pip uninstall tensorboard-plugin-wit

In [ ]:
%cd '/content/gdrive/MyDrive/Tensorflow/workspace/training_demo'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/{model_name}


# Model training

In [ ]:
%cd /content/gdrive/MyDrive/Tensorflow/workspace/training_demo
!python model_main.py --model_dir=/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/models/ssd_inception_v2_coco_2018_01_28 --pipeline_config_path=/content/gdrive/MyDrive/Tensorflow/workspace/training_demo/models/ssd_inception_v2_coco_2018_01_28/ssd_inception_v2_coco.config